In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from skimage.feature import peak_local_max
import mrcfile
import napari
import starfile

In [3]:
eman2_directory = Path("/mnt/scratch/ribosomes/wws_EGFcontrol/eman2")
tomo_directory = eman2_directory / Path("tomograms/")
segment_directory = eman2_directory / Path("segmentations/")

# Construct dictionary
ts_dict = {}

for tomo_path in sorted(tomo_directory.glob("*.mrc")):
    ts_name = tomo_path.stem.split("_")[0]
    ts_dict[ts_name] = {}
    segment_path = segment_directory / Path(f"{ts_name}_rec_corrected__ribosomes_good_seg.mrc")

    if tomo_path.is_file() and segment_path.is_file():
        ts_dict[ts_name]["tomo_path"] = tomo_path
        ts_dict[ts_name]["segment_path"] = segment_directory / Path(f"{ts_name}_rec_corrected__ribosomes_good_seg.mrc") 
        ts_dict[ts_name]["peaks"] = None


In [4]:
tomo_bin_factor = 10
peak_separation = 10 # pixels
unbinned_pixel_size = 1.0825
print(f"Peaks will be at least {peak_separation * tomo_bin_factor * unbinned_pixel_size / 10.0} nm apart.")

ts_index = 0

Peaks will be at least 10.825 nm apart.


In [8]:
threshold_rel = 0.1
threshold_abs = 1
peak_files = segment_directory.glob("*.peaks")

to_load = ts_dict.keys()

for ts_name in to_load:
    if Path(f"{ts_name}_{threshold_abs}abs.peaks").absolute() not in peak_files:
        ts_dict[ts_name]["tomogram_mrc"] = mrcfile.read(ts_dict[ts_name]["tomo_path"])
        ts_dict[ts_name]["segment_mrc"]= mrcfile.read(ts_dict[ts_name]["segment_path"])
        peaks = peak_local_max(
            ts_dict[ts_name]["segment_mrc"],
            min_distance=peak_separation, 
            threshold_abs=threshold_abs, 
            threshold_rel=threshold_rel,
            )
        ts_dict[ts_name]["peaks"] = peaks
        np.savetxt(segment_directory / f"{ts_name}_{threshold_abs}abs.peaks", peaks, delimiter="\t", fmt="%d")
    ts_index += 1


In [12]:
viewer = napari.Viewer(ndisplay=3)
for ts_name in to_load:
    viewer.add_image(
        ts_dict[ts_name]["tomogram_mrc"], 
        name=f"{ts_name} tomogram",
        depiction="plane",
        blending="translucent",
        contrast_limits=[-15, 15],
        visible=False,
    )
    viewer.add_image(
        ts_dict[ts_name]["segment_mrc"],
        name=f"{ts_name} segment",
        depiction="volume",
        blending="translucent",
        colormap="gist_earth",
        visible=False,
    )
    viewer.add_points(
        ts_dict[ts_name]["peaks"], 
        name=f"{ts_name} peaks",
        blending="translucent",
        edge_color="red",
        edge_width=0.2, 
        visible=False
    )


In [ ]:
# Convert the calculated peak coordinates from binned space to unbinned space. Star files for RELION required unbinned coordinates.
rln_coordinates = ["rlnCoordinateZ", "rlnCoordinateY", "rlnCoordinateX"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]

#rln_tomoname = "ts002"
#peak_coords_df = pd.DataFrame(peak_coords,columns=rln_coordinates)
#unbinned_peak_coords_df = pd.DataFrame()
#for coordinate in rln_coordinates:
#    unbinned_peak_coords_df[coordinate] = tomo_bin_factor * peak_coords_df[coordinate]
#for angle in rln_angles:
#    unbinned_peak_coords_df[angle] = 0.0
#unbinned_peak_coords_df["rlnTomoName"] = rln_tomoname
#unbinned_peak_coords_df.head(3)
#
# Create empty pd columns and append entries for every tomogram in tomos
name_i = 0
unbinned_peak_coords_dict = {}

import numpy as np

for ts_name in to_load:
    unbinned_peak_coords_dict[ts_name] = pd.DataFrame(ts_dict[ts_name]["peaks"],columns=rln_coordinates) * tomo_bin_factor
    for angle in rln_angles:
        random_angles = np.random.randint(0,179,size=len(unbinned_peak_coords_dict[ts_name].index))
        unbinned_peak_coords_dict[ts_name][angle] = random_angles
    unbinned_peak_coords_dict[ts_name]["rlnTomoName"] = ts_name
    name_i+=1

    write_path = segment_directory / f"particles_{ts_name}.star"
    starfile.write(unbinned_peak_coords_dict[ts_name], write_path, overwrite=True)

In [ ]:
# Initialize the df that will be written to the star file
particles_df = pd.DataFrame(columns=rln_coordinates+rln_angles+["rlnTomoName"])

# Concatenate particle dfs from each tomogram into one
for ts_name in unbinned_peak_coords_dict.keys():
    particles_df = pd.concat([particles_df, unbinned_peak_coords_dict[ts_name]], ignore_index=True)

# Write the star file
write_path = segment_directory / f"particles_abs{threshold_abs}rel{threshold_rel}.star"
starfile.write(particles_df, write_path, overwrite=True)